In [1]:
import sqlite3
import pandas as pd
import yaml

In [12]:
connection = sqlite3.connect("../data/fpl.db")
raw_data = pd.read_sql_query("SELECT * FROM raw_team_match_log where comp='Premier League'", connection)
raw_data

,season,team,date,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,link
0,2021-2022,Manchester City,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,2.0,1.0,65.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
1,2021-2022,Manchester City,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,2.7,0.1,67.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
2,2021-2022,Manchester City,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,4.0,0.2,80.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
3,2021-2022,Manchester City,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,3.3,0.6,61.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
4,2021-2022,Manchester City,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,1.2,0.5,64.0,https://fbref.com/en/squads/b8fd03ef/2021-2022...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7595,2012-2013,QPR,2013-04-20,None,Premier League,Matchweek 34,Sat,Home,L,0,2,Stoke City,NaN,NaN,NaN,https://fbref.com/en/squads/a757999c/2012-2013...
7596,2012-2013,QPR,2013-04-28,None,Premier League,Matchweek 35,Sun,Away,D,0,0,Reading,NaN,NaN,NaN,https://fbref.com/en/squads/a757999c/2012-2013...
7597,2012-2013,QPR,2013-05-04,None,Premier League,Matchweek 36,Sat,Home,L,0,1,Arsenal,NaN,NaN,NaN,https://fbref.com/en/squads/a757999c/2012-2013...
7598,2012-2013,QPR,2013-05-12,None,Premier League,Matchweek 37,Sun,Home,L,1,2,Newcastle Utd,NaN,NaN,NaN,https://fbref.com/en/squads/a757999c/2012-2013...


In [47]:
import itertools

def calculate_points(row):
    if row["gf"] > row["ga"]:
        return 3
    elif row["gf"] == row["ga"]:
        return 1
    else:
        return 0


def calculate_daily_rank(date_data:pd.DataFrame)->pd.Series:
    date_data = date_data.sort_values(["pts_b4_match", "team"], ascending=[False, True])
    date_data["rank_b4_match"] = date_data.reset_index().index + 1
    return date_data   
    
    
def calculate_rank_from_points(points_data:pd.DataFrame)->pd.DataFrame:
    points_data = pd.DataFrame(columns=["season", "team", "date", "pts_b4_match", "rank_b4_match"])
    for season, season_data in points_data.groupby("season"):
        team_date = pd.DataFrame(list(
            itertools.product(
                [season],
                season_data["team"].unique(),
                season_data['date'].unique()
            )
        ), columns=["season", "team", "date"])
        season_data["matchday"] = True
        fill_dates = team_date.merge(season_data, on=["season", "team", "date"], how="left")
        fill_dates = fill_dates.sort_values('date')
        fill_dates["pts_b4_match"] = fill_dates.groupby('team')['pts_b4_match'].ffill().fillna(0)
        fill_dates = fill_dates.groupby('date').apply(calculate_daily_rank).reset_index(drop=True)
        fill_dates = fill_dates.loc[fill_dates["matchday"]==True].drop("matchday",axis=1)
        points_data = pd.concat([points_data,fill_dates])
        
    points_data = points_data.reset_index()
    return points_data


def calculate_pts_data(data: pd.DataFrame) -> pd.DataFrame:
    pts_data = data.copy()

    pts_data["points"] = pts_data.apply(calculate_points, axis=1)
    pts_data["total_points"] = (pts_data.groupby(["season", "team"])["points"]).shift(1)
    pts_data["pts_b4_match"] = (
        pts_data.groupby(["season", "team"])["total_points"].cumsum().fillna(0)
    )
    columns_to_drop = [col for col in pts_data.columns if col not in ['date', 'team', 'pts_b4_match', 'season']]
    pts_data = pts_data.drop(columns=columns_to_drop)
    
    return pts_data

In [113]:
data = raw_data.copy()

pts_data = calculate_pts_data(data)

import itertools

def calculate_rank_b4_match(date_data:pd.DataFrame)->pd.Series:
    date_data = date_data.sort_values(["pts_b4_match", "team"], ascending=[False, True])
    date_data["rank_b4_match"] = date_data.reset_index().index + 1
    return date_data


points_data = pd.DataFrame(columns=["season", "team", "date", "pts_b4_match", "rank_b4_match"])
for season, season_data in pts_data.groupby("season"):
    team_date = pd.DataFrame(list(
        itertools.product(
            [season],
            season_data["team"].unique(),
            season_data['date'].unique()
        )
    ), columns=["season", "team", "date"])
    season_data["matchday"] = True
    fill_dates = team_date.merge(season_data, on=["season", "team", "date"], how="left")
    fill_dates = fill_dates.sort_values('date')
    fill_dates["pts_b4_match"] = fill_dates.groupby('team')['pts_b4_match'].ffill().fillna(0)
    fill_dates = fill_dates.groupby('date').apply(calculate_rank_b4_match).reset_index(drop=True)
    fill_dates = fill_dates.loc[fill_dates["matchday"]==True].drop("matchday",axis=1)
    points_data = pd.concat([points_data,fill_dates])
    
points_data = points_data.reset_index()

points_data.query("team=='Arsenal'")
# pts_data.query("season=='2012-2013' and team=='QPR'")

,index,season,team,date,pts_b4_match,rank_b4_match
0,0,2012-2013,Arsenal,2012-08-18,0.0,1
37,108,2012-2013,Arsenal,2012-08-26,1.0,9
54,150,2012-2013,Arsenal,2012-09-02,2.0,11
64,167,2012-2013,Arsenal,2012-09-15,5.0,8
91,242,2012-2013,Arsenal,2012-09-23,8.0,3
...,...,...,...,...,...,...
7517,2223,2021-2022,Arsenal,2022-05-01,60.0,4
7537,2283,2021-2022,Arsenal,2022-05-08,63.0,4
7554,2343,2021-2022,Arsenal,2022-05-12,66.0,4
7570,2383,2021-2022,Arsenal,2022-05-16,66.0,4
